# Pre-process MSUPRP sample data for BATools

## Information about the dataset

The data comes from https://github.com/steibelj/gwaR, an example dataset for gwas study for MSUPRP dataset. The complete dataset can be also found at https://github.com/steibelj/GWA_growth.

The dataset comes as `gpData` from the `synbreed` R package.

This example will demonstrate how to create pedigree based relationship matrix and adaptive window based LD from the dataset.

Finally, we'll create the data file for BATools.

### 0. Before you start
Make sure you have the following R package installed: `dplyr`,`MCMCglmm`, `Matrix`

### 1. Explore the data 

In [1]:
load("MSUPRP_sample.RData")
class(MSUPRP_sample)
str(MSUPRP_sample)

[1] "gpData"

List of 7
 $ covar      :'data.frame':	253 obs. of  9 variables:
  ..$ id        : chr [1:253] "6070" "6071" "6086" "6088" ...
  ..$ phenotyped: logi [1:253] FALSE FALSE FALSE FALSE FALSE FALSE ...
  ..$ genotyped : logi [1:253] TRUE TRUE TRUE TRUE TRUE TRUE ...
  ..$ family    : logi [1:253] NA NA NA NA NA NA ...
  ..$ sex       : Factor w/ 2 levels "F","M": NA NA NA NA NA NA NA NA NA NA ...
  ..$ litter    : Factor w/ 142 levels "1","2","3","4",..: NA NA NA NA NA NA NA NA NA NA ...
  ..$ slgdt_cd  : Factor w/ 33 levels "1","2","3","4",..: NA NA NA NA NA NA NA NA NA NA ...
  ..$ age_slg   : int [1:253] NA NA NA NA NA NA NA NA NA NA ...
  ..$ car_wt    : num [1:253] NA NA NA NA NA NA NA NA NA NA ...
 $ pheno      : num [1:194, 1:3, 1] 5.22 5.19 5.44 5.36 5.25 5.29 5.44 5.26 5.35 5.39 ...
  ..- attr(*, "dimnames")=List of 3
  .. ..$ : chr [1:194] "1034" "1036" "1041" "1049" ...
  .. ..$ : chr [1:3] "ph_24h" "temp_24h" "driploss"
  .. ..$ : NULL
 $ geno       : num [1:251, 1:20597] 0 0 1

### 2. Merge phenotype and covar to a dataframe
Commonly, using dataframe or datatable to manage all phenotypes are easier. So we'll merge the two together

In [4]:
library(dplyr)
tmp<-data.frame(MSUPRP_sample$pheno)
tmp$id<-rownames(tmp)
colnames(tmp)=c("ph_24h"," temp_24h","driploss","id")
pheno<-left_join(tmp,MSUPRP_sample$covar,by="id")
head(pheno)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



ph_24h,temp_24h,driploss,id,phenotyped,genotyped,family,sex,litter,slgdt_cd,age_slg,car_wt
5.22,1.6,0.95,1034,TRUE,TRUE,NA,F,5,2,160,85.49
5.19,1.3,0.82,1036,TRUE,TRUE,NA,F,5,2,160,71.66
5.44,1.8,1.15,1041,TRUE,TRUE,NA,M,5,2,160,80.27
5.36,1.8,0.55,1049,TRUE,TRUE,NA,M,5,2,160,81.86
5.25,1.3,2.87,1058,TRUE,TRUE,NA,F,10,2,159,72.11
5.29,1.5,1.02,1060,TRUE,TRUE,NA,F,10,2,159,70.75


### 3. Check genotype matrix

In [5]:
geno<-MSUPRP_sample$geno
dim(geno)
geno[1:5,1:10]

[1]   251 20597

,MARC0044150,ASGA0000014,H3GA0000032,ASGA0000047,H3GA0000057,H3GA0000082,ALGA0000251,MARC0049965,ASGA0101285,ASGA0102762
1034,0,2,0,2,2,0,1,2,2,1
1036,0,2,0,2,2,0,1,2,2,1
1041,1,2,1,2,2,1,0,2,2,0
1049,1,2,1,2,2,1,0,2,2,0
1058,1,2,1,1,1,0,0,2,2,0


In BATools, the `id` will be matched with the `colname` of the genotype matrix, make sure you have genotype for all the individuals unless you're using single-step approach.


In [6]:
genof0<-geno[which(as.numeric(rownames(geno))>6000),]
dim(genof0)
maf.f0<-colMeans(genof0,na.rm = T)/2
summary(maf.f0)

[1]    20 20597

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0250  0.3000  0.5250  0.5102  0.7000  0.9750 

### 4. Create pedigree based additive genomic relationship matrix

In [14]:
ped<-MSUPRP_sample$pedigree
head(ped)
source("tools.R")
library(pedigree)
colnames(ped)=c("id","father","mother","generation","sex")
ped0=Pedigree(ped,unknown = 0)
Ainv<-MCMCglmm::inverseA(pedigree=ped0[,1:3],nodes="ALL")$Ainv
A=Matrix::solve(Ainv,sparse=TRUE,tol=1e-16)
A[which(A<1e-6)]=0
colnames(A)=colnames(Ainv)=ped0$id
A[1:5,1:5]


ID,Par1,Par2,gener,sex
6070,0,0,0,0
6071,0,0,0,0
6086,0,0,0,0
6088,0,0,0,0
6092,0,0,0,0
6323,0,0,0,1


Warning message in if (attr(pedigree, "class") == "phylo") {:
"the condition has length > 1 and only the first element will be used"

5 x 5 sparse Matrix of class "dgCMatrix"
     6070 6071 6086 6088 6092
[1,]    1    .    .    .    .
[2,]    .    1    .    .    .
[3,]    .    .    1    .    .
[4,]    .    .    .    1    .
[5,]    .    .    .    .    1

### 5. Building map with adaptive window based on LD
Install `BALD` package from http://www.math-evry.cnrs.fr/logiciels/bald. On windows, `RTools` need to be pre-installed https://cran.r-project.org/bin/windows/Rtools/ and add your `RTools` path to enviroment variable. But linux or Mac are recommended since `BALD` is not available at CRAN. 

Installing these packages can be complicated, if you have any question, send me an email at chench57{at}msu.edu

In [15]:
source("https://bioconductor.org/biocLite.R")
biocLite("chopsticks")
biocLite("snpStats")
biocLite("ROC")
install.packages(c("LDheatmap","quadrupen", "ROC", "grplasso","snpStats"))


Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Bioconductor version 3.4 (BiocInstaller 1.24.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.4 (BiocInstaller 1.24.0), R 3.3.2 (2016-10-31).
Installing package(s) 'chopsticks'
also installing the dependency 'survival'

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'R6', 'digest', 'evaluate', 'jsonlite', 'memoise', 'pbdZMQ',
  'repr', 'stringi', 'stringr'
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.4 (BiocInstaller 1.24.0), R 3.3.2 (2016-10-31).
Installing package(s) 'snpStats'
also installing the dependencies 'BiocGenerics', 'zlibbioc'

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'R6', 'digest', 'evaluate', 'jsonlite', 'memoise', 'pbdZMQ',
  '

In [16]:
install.packages("matrixStats")
system("wget http://www.math-evry.cnrs.fr/_media/logiciels/bald_0.2.1.tar.gz")
system("R CMD INSTALL bald_0.2.1.tar.gz")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


Ok, now we can start to use BALD for obtain the LD based window

In [17]:
library(BALD)
map<-MSUPRP_sample$map
head(map)
map=MSUPRP_sample$map
chrs=list()
for(i in 1:max(map$chr)){
  ii=which(map$chr==i)
  chrs[[i]]=geno[,ii]
}

#This will take couple hours, suggest you run it in parallel in high-performance computing (HPC) platform, which was what I did. 
if(F){
for(i in 1:length(chrs)){
  Z=chr[[i]]+1
  p=dim(Z)[2]
  gapS <- gapStatistic(Z, min.nc=2, max.nc=p-1, B=50)
  gapS$best.k
  adaptiveWindows[[i]] <- cutree(gapS$tree, gapS$best.k)
}
}
#After running this througth, I save the file in `bald.RData`

Loading required package: grid

Attaching package: 'BALD'

The following object is masked from 'package:dplyr':

    select



,chr,pos
MARC0044150,1,286933
ASGA0000014,1,342481
H3GA0000032,1,573088
ASGA0000047,1,813652
H3GA0000057,1,1220227
H3GA0000082,1,1843489


Let's then re-load the file we computed on HPC

In [ ]:
load("bald.RData")
idw<-adaptiveWindows[[1]]
for(i in 2:length(adaptiveWindows)){
  tmp<-max(idw)
  idw<-c(idw,adaptiveWindows[[i]]+tmp)
}
map$idw=idw

Now let’s save all the files

In [ ]:
PigA<-A
PigAinv<-Ainv
PigM<-geno
PigMap<-map
PigPheno<-pheno %>% 
          filter(!is.na(phenotyped)) %>% 
          dplyr::select(driploss,id,sex,litter,slgdt_cd,age_slg,car_wt)
PigPed<-ped0
PigAlleleFreq<-freq.f0
save(PigA,PigAinv,PigM,PigMap,PigPheno,PigPed,PigAlleleFreq,file="Pig.RData")